In [ ]:
from depmap_omics_upload import tracker
import pandas as pd

In [ ]:
mytracker = tracker.SampleTracker()
seq_table = mytracker.add_model_cols_to_seqtable()
seq_table = seq_table[(~seq_table.blacklist) & (seq_table.expected_type.isin(["rna", "wgs", "wes"]))]

In [ ]:
pr_table = mytracker.add_model_cols_to_prtable()

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

OmicsDefaultModelProfiles = tc.get(name='internal-22q4-56d4', version=90, file='OmicsDefaultModelProfiles')

In [ ]:
OmicsDefaultModelProfiles

In [ ]:
seq_table

In [ ]:
lines_to_check = pd.read_csv("check_for_dbgap.csv", index_col=0)
lines_to_check['rna_bam'] = pd.NA
lines_to_check['wgs_bam'] = pd.NA
lines_to_check['wes_bam'] = pd.NA

In [ ]:
from google.cloud import storage
storage_client = storage.Client()
bucket_name = 'cclebams'
bucket = storage_client.bucket(bucket_name)

In [ ]:
for i in lines_to_check.index:
    rna = seq_table[(seq_table.ModelID == i) & (seq_table.expected_type == "rna")]
    if len(rna) != 0:
        rna_latestid = rna["version"].idxmax()
        bam_path = rna.loc[rna_latestid, "bam_filepath"]
        relative_path = "/".join(bam_path.split("/")[3:])
        stats = storage.Blob(bucket=bucket, name=relative_path).exists(storage_client)
        if stats == False:
            print(bam_path + " is missing")
        else:
            lines_to_check.loc[i, 'rna_bam'] = bam_path
    wgs = seq_table[(seq_table.ModelID == i) & (seq_table.expected_type == "wgs")]
    if len(wgs) != 0:
        wgs_latestid = wgs["version"].idxmax()
        bam_path = wgs.loc[wgs_latestid, "bam_filepath"]
        relative_path = "/".join(bam_path.split("/")[3:])
        stats = storage.Blob(bucket=bucket, name=relative_path).exists(storage_client)
        if stats == False:
            print(bam_path + " is missing")
        else:
            lines_to_check.loc[i, 'wgs_bam'] = bam_path
    wes = seq_table[(seq_table.ModelID == i) & (seq_table.expected_type == "wes")]
    if len(wes) != 0:
        wes_latestid = wes["version"].idxmax()
        bam_path = wes.loc[wes_latestid, "bam_filepath"]
        relative_path = "/".join(bam_path.split("/")[3:])
        stats = storage.Blob(bucket=bucket, name=relative_path).exists(storage_client)
        if stats == False:
            print(bam_path + " is missing")
        else:
            lines_to_check.loc[i, 'wes_bam'] = bam_path

In [ ]:
stats

In [ ]:
relative_path

In [ ]:
storage.Blob(bucket=bucket, name='hg38_wes/CDS-4w5dFL.hg38.bam').exists(storage_client)

In [ ]:
lines_to_check.to_csv("dbgap_line_availability.csv")

In [ ]:
# check CCLF line availability

In [ ]:
cclf_lines = pd.read_csv("List of CCLF Models_potentialDMC.csv", index_col=0)

In [ ]:
model_table = mytracker.read_model_table()

In [ ]:
model_table["aliases"] = model_table.cell_line_aliases.apply(lambda x: x.split(", ") if x != None else [])

In [ ]:
aliases = set(model_table["aliases"].sum())

In [ ]:
cclf_lines["model_exists_in_gumbo"] = "No"

In [ ]:
cclf_lines.loc[cclf_lines[(cclf_lines["CCLF Publication ID"].isin(set(model_table.CellLineName))) | (cclf_lines["CCLF Publication ID"].isin(aliases))].index, "model_exists_in_gumbo"] = "Yes"

In [ ]:
missing_arxspan = cclf_lines[(cclf_lines.model_exists_in_gumbo == "Yes") & (cclf_lines["Arxspan ID"] == "-")]
for i in missing_arxspan.index.tolist():
    cclf_lines.loc[i, "model_exists_in_gumbo"] = "Cell line name exists, arxspan ID in gumbo is " + model_table[(model_table.CellLineName == cclf_lines.loc[i, "CCLF Publication ID"]) | (model_table.cell_line_aliases.str.contains(cclf_lines.loc[i, "CCLF Publication ID"]))].index.tolist()[0]

In [ ]:
ach_cclfid_mismatch = cclf_lines[(cclf_lines.model_exists_in_gumbo == "No") & (cclf_lines["Arxspan ID"] != "-")]
for i in ach_cclfid_mismatch.index.tolist():
    cclf_lines.loc[i, "model_exists_in_gumbo"] = "Arxspan ID exists, but cell line name in gumbo is " + model_table.loc[cclf_lines.loc[i, "Arxspan ID"], "CellLineName"]

In [ ]:
cclf_lines["rnaseq_profile_available"] = "No"
cclf_lines["wes_profile_available"] = "No"

cclf_lines["rnaseq_profile_release_date"] = ""
cclf_lines["wes_profile_release_date"] = ""

for i in cclf_lines[cclf_lines["model_exists_in_gumbo"] == "Yes"].index.tolist():
    if cclf_lines.loc[i, "Arxspan ID"] != "-":
        rna_prs = pr_table[(pr_table.Datatype == "rna") & (pr_table.ModelID == cclf_lines.loc[i, "Arxspan ID"]) & (pr_table.BlacklistOmics != True)]
        if len(rna_prs) > 0:
            dates = rna_prs.ConsortiumReleaseDate.astype(str).tolist()
            if set(dates) == {"None"}:
                cclf_lines.loc[i, "rnaseq_profile_release_date"] = "Not assigned"
            else:
                if "None" in dates:
                    dates.remove("None")
                cclf_lines.loc[i, "rnaseq_profile_release_date"] = dates[0]
            cclf_lines.loc[i, "rnaseq_profile_available"] = "Yes"
        wes_prs = pr_table[(pr_table.Datatype == "wes") & (pr_table.ModelID == cclf_lines.loc[i, "Arxspan ID"]) & (pr_table.BlacklistOmics != True)]
        if len(wes_prs) > 0:
            dates = wes_prs.ConsortiumReleaseDate.astype(str).tolist()
            if set(dates) == {"None"}:
                cclf_lines.loc[i, "wes_profile_release_date"] = "Not assigned"
            else:
                if "None" in dates:
                    dates.remove("None")
                cclf_lines.loc[i, "wes_profile_release_date"] = dates[0]
            cclf_lines.loc[i, "wes_profile_available"] = "Yes"
    else:
        arxspan = model_table[(model_table.CellLineName == cclf_lines.loc[i, "CCLF Publication ID"]) | (model_table.cell_line_aliases.str.contains(cclf_lines.loc[i, "CCLF Publication ID"]))].index.tolist()[0]
        rna_prs = pr_table[(pr_table.Datatype == "rna") & (pr_table.ModelID == arxspan) & (pr_table.BlacklistOmics != True)]
        if len(rna_prs) > 0:
            dates = rna_prs.ConsortiumReleaseDate.astype(str).tolist()
            if set(dates) == {"None"}:
                cclf_lines.loc[i, "rnaseq_profile_release_date"] = "Not assigned"
            else:
                if "None" in dates:
                    dates.remove("None")
                cclf_lines.loc[i, "rnaseq_profile_release_date"] = dates[0]
            cclf_lines.loc[i, "rnaseq_profile_available"] = "Yes"
        wes_prs = pr_table[(pr_table.Datatype == "wes") & (pr_table.ModelID == arxspan) & (pr_table.BlacklistOmics != True)]
        if len(wes_prs) > 0:
            dates = wes_prs.ConsortiumReleaseDate.astype(str).tolist()
            if set(dates) == {"None"}:
                cclf_lines.loc[i, "wes_profile_release_date"] = "Not assigned"
            else:
                if "None" in dates:
                    dates.remove("None")
                cclf_lines.loc[i, "wes_profile_release_date"] = dates[0]
            cclf_lines.loc[i, "wes_profile_available"] = "Yes"

In [ ]:
for i in cclf_lines[(cclf_lines["Arxspan ID"] != "-") & (cclf_lines["model_exists_in_gumbo"] != "Yes")].index.tolist():
    rna_prs = pr_table[(pr_table.Datatype == "rna") & (pr_table.ModelID == cclf_lines.loc[i, "Arxspan ID"]) & (pr_table.BlacklistOmics != True)]
    if len(rna_prs) > 0:
        dates = rna_prs.ConsortiumReleaseDate.astype(str).tolist()
        if set(dates) == {"None"}:
            cclf_lines.loc[i, "rnaseq_profile_release_date"] = "Not assigned"
        else:
            if "None" in dates:
                dates.remove("None")
            cclf_lines.loc[i, "rnaseq_profile_release_date"] = dates[0]
        cclf_lines.loc[i, "rnaseq_profile_available"] = "Yes"
    wes_prs = pr_table[(pr_table.Datatype == "wes") & (pr_table.ModelID == cclf_lines.loc[i, "Arxspan ID"]) & (pr_table.BlacklistOmics != True)]
    if len(wes_prs) > 0:
        dates = wes_prs.ConsortiumReleaseDate.astype(str).tolist()
        if set(dates) == {"None"}:
            cclf_lines.loc[i, "wes_profile_release_date"] = "Not assigned"
        else:
            if "None" in dates:
                dates.remove("None")
            cclf_lines.loc[i, "wes_profile_release_date"] = dates[0]
        cclf_lines.loc[i, "wes_profile_available"] = "Yes"

In [ ]:
cclf_lines[cclf_lines.rnaseq_profile_available == "Yes"]

In [ ]:
cclf_lines.to_csv("CCLF_availability.csv")

In [ ]:
model_table.columns